 - Batting Team
 - Bowling Team
 - Score
 - Overs Completed
 - Wickets out
 - Run in last 5 over
 - Venue

In [80]:
import pandas as pd
import pickle
import numpy as np

In [81]:
matches = pickle.load(open('dataset_level1.pkl','rb'))

In [82]:
matches.head()

,innings,info.city,info.outcome.winner,info.player_of_match,info.teams,info.venue,match_id
0,[{'1st innings': {'team': 'Sunrisers Hyderabad...,Hyderabad,Sunrisers Hyderabad,[Yuvraj Singh],"[Sunrisers Hyderabad, Royal Challengers Bangal...","Rajiv Gandhi International Stadium, Uppal",1
0,"[{'1st innings': {'team': 'Mumbai Indians', 'd...",Pune,Rising Pune Supergiant,[SPD Smith],"[Rising Pune Supergiant, Mumbai Indians]",Maharashtra Cricket Association Stadium,2
0,"[{'1st innings': {'team': 'Gujarat Lions', 'de...",Rajkot,Kolkata Knight Riders,[CA Lynn],"[Gujarat Lions, Kolkata Knight Riders]",Saurashtra Cricket Association Stadium,3
0,[{'1st innings': {'team': 'Rising Pune Supergi...,Indore,Kings XI Punjab,[GJ Maxwell],"[Kings XI Punjab, Rising Pune Supergiant]",Holkar Cricket Stadium,4
0,[{'1st innings': {'team': 'Royal Challengers B...,Bengaluru,Royal Challengers Bangalore,[KM Jadhav],"[Royal Challengers Bangalore, Delhi Daredevils]",M.Chinnaswamy Stadium,5


In [4]:
#team(matches['innings'])

In [86]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    non_striker=[]
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    winner=[]
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            non_striker.append(ball[key]['non_striker'])
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            winner.append(row['info.outcome.winner'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'non_striker':non_striker,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue,
            'winner':winner
        })
    delivery_df = delivery_df.append(loop_df)

In [87]:
delivery_df.head()

,match_id,teams,batting_team,ball,batsman,non_striker,bowler,runs,player_dismissed,city,venue,winner
0,2,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad
1,2,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad
2,2,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad
3,2,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad
4,2,"[Sunrisers Hyderabad, Royal Challengers Bangal...",Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,0,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad


In [88]:
def bowl(row):
    for i in row['teams']:
        if i != row['batting_team']:
            return i

In [89]:
delivery_df['bowling_team']=delivery_df.apply(bowl,axis=1)
    

In [90]:
delivery_df.isnull().sum()

match_id               0
teams                  0
batting_team           0
ball                   0
batsman                0
non_striker            0
bowler                 0
runs                   0
player_dismissed       0
city                6343
venue                  0
winner              1957
bowling_team           0
dtype: int64

In [91]:
#delivery_df['city'] = delivery_df.fillna(delivery_df.loc[delivery_df['city'].isna()]['venue'])

In [92]:
delivery_df.loc[delivery_df['city'].isna()]['venue'].unique()

array(['Dubai International Cricket Stadium', 'Sharjah Cricket Stadium'],
      dtype=object)

In [93]:
temp_2 = delivery_df.loc[delivery_df['city'].isna()]['venue'].str.split(" ",expand=True)[0]
temp_2 = pd.Series(temp_2)

In [94]:
cities = np.where(delivery_df['city'].isnull(),delivery_df['venue'].str.split(" ",expand=True)[0],delivery_df['city'])
delivery_df['city'] = cities

In [95]:
delivery_df.isnull().sum()

match_id               0
teams                  0
batting_team           0
ball                   0
batsman                0
non_striker            0
bowler                 0
runs                   0
player_dismissed       0
city                   0
venue                  0
winner              1957
bowling_team           0
dtype: int64

In [96]:
delivery_df.drop(columns=['teams','venue'],inplace=True)

In [97]:
delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['runs']

In [98]:
delivery_df.head(50)

,match_id,batting_team,ball,batsman,non_striker,bowler,runs,player_dismissed,city,winner,bowling_team,current_score
0,2,Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0
1,2,Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0
2,2,Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4
3,2,Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4
4,2,Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6
5,2,Sunrisers Hyderabad,0.6,S Dhawan,DA Warner,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6
6,2,Sunrisers Hyderabad,0.7,S Dhawan,DA Warner,TS Mills,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7
7,2,Sunrisers Hyderabad,1.1,S Dhawan,DA Warner,A Choudhary,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8
8,2,Sunrisers Hyderabad,1.2,DA Warner,S Dhawan,A Choudhary,4,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12
9,2,Sunrisers Hyderabad,1.3,DA Warner,S Dhawan,A Choudhary,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13


In [99]:
for i in delivery_df['player_dismissed']:
    if i!='0':
        delivery_df['player_dismissed'] = delivery_df['player_dismissed'].replace(i,1)

In [100]:
delivery_df.head(15)

,match_id,batting_team,ball,batsman,non_striker,bowler,runs,player_dismissed,city,winner,bowling_team,current_score
0,2,Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0
1,2,Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0
2,2,Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4
3,2,Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4
4,2,Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6
5,2,Sunrisers Hyderabad,0.6,S Dhawan,DA Warner,TS Mills,0,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6
6,2,Sunrisers Hyderabad,0.7,S Dhawan,DA Warner,TS Mills,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7
7,2,Sunrisers Hyderabad,1.1,S Dhawan,DA Warner,A Choudhary,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8
8,2,Sunrisers Hyderabad,1.2,DA Warner,S Dhawan,A Choudhary,4,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12
9,2,Sunrisers Hyderabad,1.3,DA Warner,S Dhawan,A Choudhary,1,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13


In [101]:
delivery_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99036 entries, 0 to 125
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_id          99036 non-null  int64  
 1   batting_team      99036 non-null  object 
 2   ball              99036 non-null  float64
 3   batsman           99036 non-null  object 
 4   non_striker       99036 non-null  object 
 5   bowler            99036 non-null  object 
 6   runs              99036 non-null  int64  
 7   player_dismissed  99036 non-null  object 
 8   city              99036 non-null  object 
 9   winner            97079 non-null  object 
 10  bowling_team      99036 non-null  object 
 11  current_score     99036 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 9.8+ MB


In [102]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')

In [103]:
delivery_df['player_dismissed'] = delivery_df.groupby('match_id').cumsum()['player_dismissed']

In [104]:
delivery_df['wickets_left'] = 10 - delivery_df['player_dismissed']

In [105]:
delivery_df.drop(columns='player_dismissed',inplace=True)

In [106]:
delivery_df.head(30)

,match_id,batting_team,ball,batsman,non_striker,bowler,runs,city,winner,bowling_team,current_score,wickets_left
0,2,Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10
1,2,Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10
2,2,Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10
3,2,Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10
4,2,Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10
5,2,Sunrisers Hyderabad,0.6,S Dhawan,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10
6,2,Sunrisers Hyderabad,0.7,S Dhawan,DA Warner,TS Mills,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7,10
7,2,Sunrisers Hyderabad,1.1,S Dhawan,DA Warner,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8,10
8,2,Sunrisers Hyderabad,1.2,DA Warner,S Dhawan,A Choudhary,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12,10
9,2,Sunrisers Hyderabad,1.3,DA Warner,S Dhawan,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13,10


In [107]:
over = []
ball_no = []
for i in delivery_df['ball']:
    temp_3 = str(i).split(".")[0]
    temp_4 = str(i).split(".")[1]
    ball_no.append(temp_4)
    over.append(temp_3)
delivery_df['over']= over
delivery_df['ball_no'] = ball_no
delivery_df['balls_bowled'] = (delivery_df['over'].astype('int')*6) + (delivery_df['ball_no'].astype('int'))
delivery_df['balls_left'] = 120-delivery_df['balls_bowled']
delivery_df['balls_left'].apply(lambda x:0 if x<0 else x)
#[0 if delivery_df['balls_left']<0 else 1]

0      119
1      118
2      117
3      116
4      115
      ... 
121      4
122      3
123      2
124      1
125      0
Name: balls_left, Length: 99036, dtype: int64

In [108]:
delivery_df.head(10)

,match_id,batting_team,ball,batsman,non_striker,bowler,runs,city,winner,bowling_team,current_score,wickets_left,over,ball_no,balls_bowled,balls_left
0,2,Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,1,1,119
1,2,Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,2,2,118
2,2,Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,3,3,117
3,2,Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,4,4,116
4,2,Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,5,5,115
5,2,Sunrisers Hyderabad,0.6,S Dhawan,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,6,6,114
6,2,Sunrisers Hyderabad,0.7,S Dhawan,DA Warner,TS Mills,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7,10,0,7,7,113
7,2,Sunrisers Hyderabad,1.1,S Dhawan,DA Warner,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8,10,1,1,7,113
8,2,Sunrisers Hyderabad,1.2,DA Warner,S Dhawan,A Choudhary,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12,10,1,2,8,112
9,2,Sunrisers Hyderabad,1.3,DA Warner,S Dhawan,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13,10,1,3,9,111


In [109]:
delivery_df['over'].value_counts()

0     5066
1     5065
2     5011
3     5006
4     5002
5     4997
6     4980
7     4956
9     4949
8     4947
14    4936
17    4932
16    4927
12    4921
15    4918
11    4915
10    4915
13    4904
18    4899
19    4790
Name: over, dtype: int64

In [110]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

In [111]:
delivery_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99036 entries, 0 to 125
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   match_id       99036 non-null  int64  
 1   batting_team   99036 non-null  object 
 2   ball           99036 non-null  float64
 3   batsman        99036 non-null  object 
 4   non_striker    99036 non-null  object 
 5   bowler         99036 non-null  object 
 6   runs           99036 non-null  int64  
 7   city           99036 non-null  object 
 8   winner         97079 non-null  object 
 9   bowling_team   99036 non-null  object 
 10  current_score  99036 non-null  int64  
 11  wickets_left   99036 non-null  int32  
 12  over           99036 non-null  object 
 13  ball_no        99036 non-null  object 
 14  balls_bowled   99036 non-null  int32  
 15  balls_left     99036 non-null  int32  
dtypes: float64(1), int32(3), int64(3), object(9)
memory usage: 11.7+ MB


In [112]:
delivery_df['over'] = delivery_df['over'].astype('int')
delivery_df['ball_no'] = delivery_df['ball_no'].astype('int')

In [113]:
delivery_df.head(50)

,match_id,batting_team,ball,batsman,non_striker,bowler,runs,city,winner,bowling_team,current_score,wickets_left,over,ball_no,balls_bowled,balls_left
0,2,Sunrisers Hyderabad,0.1,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,1,1,119
1,2,Sunrisers Hyderabad,0.2,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,2,2,118
2,2,Sunrisers Hyderabad,0.3,DA Warner,S Dhawan,TS Mills,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,3,3,117
3,2,Sunrisers Hyderabad,0.4,DA Warner,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,4,4,116
4,2,Sunrisers Hyderabad,0.5,DA Warner,S Dhawan,TS Mills,2,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,5,5,115
5,2,Sunrisers Hyderabad,0.6,S Dhawan,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,6,6,114
6,2,Sunrisers Hyderabad,0.7,S Dhawan,DA Warner,TS Mills,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7,10,0,7,7,113
7,2,Sunrisers Hyderabad,1.1,S Dhawan,DA Warner,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8,10,1,1,7,113
8,2,Sunrisers Hyderabad,1.2,DA Warner,S Dhawan,A Choudhary,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12,10,1,2,8,112
9,2,Sunrisers Hyderabad,1.3,DA Warner,S Dhawan,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13,10,1,3,9,111


In [114]:
teams = ['Sunrisers Hyderabad', 'Mumbai Indians','Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Rajasthan Royals', 'Chennai Super Kings', 'Delhi Capitals',]

In [115]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [116]:
delivery_df.shape

(76438, 16)

In [117]:
final_df = delivery_df.groupby('match_id').sum()['runs'].reset_index().merge(delivery_df,on='match_id')
final_df.drop(columns='non_striker',inplace=True)

In [118]:
final_df

,match_id,runs_x,batting_team,ball,batsman,bowler,runs_y,city,winner,bowling_team,current_score,wickets_left,over,ball_no,balls_bowled,balls_left
0,2,207,Sunrisers Hyderabad,0.1,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,1,1,119
1,2,207,Sunrisers Hyderabad,0.2,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,2,2,118
2,2,207,Sunrisers Hyderabad,0.3,DA Warner,TS Mills,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,3,3,117
3,2,207,Sunrisers Hyderabad,0.4,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,4,4,116
4,2,207,Sunrisers Hyderabad,0.5,DA Warner,TS Mills,2,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76433,817,208,Sunrisers Hyderabad,19.2,BCJ Cutting,SR Watson,6,Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,194,3,19,2,116,4
76434,817,208,Sunrisers Hyderabad,19.3,BCJ Cutting,SR Watson,6,Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,200,3,19,3,117,3
76435,817,208,Sunrisers Hyderabad,19.4,BCJ Cutting,SR Watson,1,Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,201,3,19,4,118,2
76436,817,208,Sunrisers Hyderabad,19.5,B Kumar,SR Watson,1,Bangalore,Sunrisers Hyderabad,Royal Challengers Bangalore,202,3,19,5,119,1


In [119]:
final_df['crr'] = (final_df['current_score']*6)/final_df['balls_bowled']

In [120]:
groups = final_df.groupby('match_id')
match_ids = final_df['match_id'].unique()
last_five = []
for i in match_ids:
    last_five.extend(groups.get_group(i).rolling(30).sum()['runs_y'].values.tolist())


In [121]:
final_df['last_five'] = last_five

In [122]:
final_df.head(2)

,match_id,runs_x,batting_team,ball,batsman,bowler,runs_y,city,winner,bowling_team,current_score,wickets_left,over,ball_no,balls_bowled,balls_left,crr,last_five
0,2,207,Sunrisers Hyderabad,0.1,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,1,1,119,0.0,NaN
1,2,207,Sunrisers Hyderabad,0.2,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,2,2,118,0.0,NaN


In [125]:
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0 

In [127]:
final_df['result']=final_df.apply(result,axis=1)

In [129]:
final_df['result'].value_counts()

0    42561
1    33877
Name: result, dtype: int64

In [131]:
final_df.groupby('match_id').cumsum()['runs_y']

0          0
1          0
2          4
3          4
4          6
        ... 
76433    194
76434    200
76435    201
76436    202
76437    208
Name: runs_y, Length: 76438, dtype: int64

In [137]:
final_df['rrr']=final_df['current_score']/final_df['ball']

In [140]:
final_df.head(200)

,match_id,runs_x,batting_team,ball,batsman,bowler,runs_y,city,winner,bowling_team,current_score,wickets_left,over,ball_no,balls_bowled,balls_left,crr,last_five,result,rrr
0,2,207,Sunrisers Hyderabad,0.1,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,1,1,119,0.000000,NaN,1,0.000000
1,2,207,Sunrisers Hyderabad,0.2,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,0,10,0,2,2,118,0.000000,NaN,1,0.000000
2,2,207,Sunrisers Hyderabad,0.3,DA Warner,TS Mills,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,3,3,117,8.000000,NaN,1,13.333333
3,2,207,Sunrisers Hyderabad,0.4,DA Warner,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,4,10,0,4,4,116,6.000000,NaN,1,10.000000
4,2,207,Sunrisers Hyderabad,0.5,DA Warner,TS Mills,2,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,5,5,115,7.200000,NaN,1,12.000000
5,2,207,Sunrisers Hyderabad,0.6,S Dhawan,TS Mills,0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,6,10,0,6,6,114,6.000000,NaN,1,10.000000
6,2,207,Sunrisers Hyderabad,0.7,S Dhawan,TS Mills,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,7,10,0,7,7,113,6.000000,NaN,1,10.000000
7,2,207,Sunrisers Hyderabad,1.1,S Dhawan,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,8,10,1,1,7,113,6.857143,NaN,1,7.272727
8,2,207,Sunrisers Hyderabad,1.2,DA Warner,A Choudhary,4,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,12,10,1,2,8,112,9.000000,NaN,1,10.000000
9,2,207,Sunrisers Hyderabad,1.3,DA Warner,A Choudhary,1,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,13,10,1,3,9,111,8.666667,NaN,1,10.000000


In [141]:
final_df = final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x','rrr','result']]

In [142]:
final_df.isnull().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
balls_left           0
wickets_left         0
crr                  0
last_five        18038
runs_x               0
rrr                  0
result               0
dtype: int64

In [143]:
final_df.dropna(inplace=True)

<ipython-input-143-3665e38311c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [144]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
rrr              0
result           0
dtype: int64

In [206]:
final_df = final_df.sample(final_df.shape[0])

In [207]:
categorical_cols =['batting_team','bowling_team','city']
encoded_dataframe = pd.get_dummies(final_df[categorical_cols],drop_first=True)
data_other_cols = final_df.drop(columns=categorical_cols)
data_out = pd.concat([data_other_cols,encoded_dataframe],axis=1)


In [148]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler().fit(data_out)
    
# transform the training data column
#X_train = scale.transform(X_train)
#y_train = scale.transform(y_train)
    
# transform the testing data column
#X_test = scale.transform(X_test)
scaled = scale.transform(data_out)

In [149]:
modelling_df = pd.DataFrame(scaled,columns=data_out.columns)

In [208]:
X = modelling_df.drop(columns='runs_x')
y=modelling_df['runs_x']

In [209]:
from sklearn.model_selection import train_test_split

In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [211]:
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

X_train (39128, 7)
X_test (19272, 7)
y_train (39128,)
y_test (19272,)


# Liner Regression

In [212]:
#!pip install xgboost

In [176]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

In [177]:
# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(X_train, y_train)
# Predict
y_predicted = regression_model.predict(X_test)

# model evaluation
#rmse = mean_squared_error(y_test, y_predicted)
r2 = r2_score(y_test, y_predicted)

# printing values
#print('Slope:' ,regression_model.coef_)
#print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', np.sqrt(mean_squared_error(y_test, y_predicted)))
print('Mean absolute error: ', mean_absolute_error(y_test, y_predicted))
print('R2 score: ', r2)

Root mean squared error:  17.03754037558138
Mean absolute error:  12.717947569112301
R2 score:  0.6734676105081717


In [178]:
slr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_predicted})
slr_diff.head()

,Actual value,Predicted value
2524,185,175.386102
3830,107,123.677739
8779,187,134.160478
45742,157,154.801394
39527,154,150.874146


# Logistic Regression

In [213]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [214]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),
                         ['batting_team','bowling_team','city'])
]
    ,remainder='passthrough')

In [215]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [216]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [217]:
y_pred = pipe.predict(X_test)

In [218]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.5318596928185969


# XGB Regressor

In [65]:
xgb_r = XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=101)
  
# Fitting the model
xgb_r.fit(X_train, y_train)
  
# Predict the model
pred1 = xgb_r.predict(X_test)

r2 = r2_score(y_test, pred1)
# RMSE Computation
rmse = np.sqrt(mean_squared_error(y_test, pred1))
print('Root mean squared error: ', rmse)
print('Mean absolute error: ', mean_absolute_error(y_test,pred1))
print('R2 score: ', r2)

Root mean squared error:  0.18766000312588307
Mean absolute error:  0.09600994488313656
R2 score:  0.9647767869516919


In [66]:
slr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': pred1})
slr_diff.head()

,Actual value,Predicted value
20755,-0.011936,0.119867
43960,1.827491,0.124736
56,-0.814596,-0.759728
32998,2.262265,2.296695
11570,-0.613931,-0.597848


# Random Forest Regressor

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
# create regressor object
regressor = RandomForestRegressor(random_state = 42)
  
# fit the regressor with x and y data
regressor.fit(X_train, y_train)  

RandomForestRegressor(random_state=42)

In [69]:
pred = regressor.predict(X_test)
r2 = r2_score(y_test, pred)
rmse = np.sqrt(mean_squared_error(y_test, pred))
print('Root mean squared error: ', rmse)
print('Mean absolute error: ', mean_absolute_error(y_test,pred))
print('R2 score: ', r2)

Root mean squared error:  0.22505924249782197
Mean absolute error:  0.11997831574099196
R2 score:  0.9493383609037563


In [70]:
slr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': pred})
slr_diff.head()

,Actual value,Predicted value
20755,-0.011936,0.086724
43960,1.827491,0.607449
56,-0.814596,-0.923958
32998,2.262265,2.262265
11570,-0.613931,-0.555403


In [71]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Pipeline

In [72]:
X = final_df.drop(columns='runs_x')
y=final_df['runs_x']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [74]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),
                         ['batting_team','bowling_team','city'])
]
    ,remainder='passthrough')

In [75]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=101))
])

In [76]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.969061907566834
2.747728203084272


In [77]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [78]:
final_df['city'].unique()

array(['Dubai', 'Sharjah', 'Mumbai', 'Chandigarh', 'Jaipur', 'Abu Dhabi',
       'Bengaluru', 'Port Elizabeth', 'Ranchi', 'Bangalore', 'Kolkata',
       'Centurion', 'Chennai', 'Ahmedabad', 'Cuttack', 'Dharamsala',
       'Delhi', 'Indore', 'Kimberley', 'East London', 'Hyderabad',
       'Cape Town', 'Pune', 'Johannesburg', 'Durban', 'Raipur',
       'Bloemfontein', 'Visakhapatnam'], dtype=object)

In [79]:
import xgboost
xgboost.__version__

'1.5.0'